# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

План проекта:
Изучить данны, провести предобработку.
Решить поставленные задачи.
Предложить алгоритм преобразования данных для решения задачи.
Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:
    data = pd.read_csv('C:/Users/user/Downloads/insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')
data.info()
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0


In [3]:
print(data.isna().sum())

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64
153


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные загружены, пропусков и дубликатов нет. Можно ещё изменить типы данные в столбце Возраст и Зарплата на целочисленный.

In [6]:
data['Возраст'] = pd.to_numeric(data['Возраст'], downcast='integer')
data['Зарплата'] = pd.to_numeric(data['Зарплата'], downcast='integer')

Корреляционный анализ: целевой признак имеет сильную корреляцию с возрастом.

In [7]:
corr_matrix = data.corr()
print(corr_matrix)

                        Пол   Возраст  Зарплата  Члены семьи  \
Пол                1.000000  0.002074  0.014910    -0.008991   
Возраст            0.002074  1.000000 -0.019093    -0.006692   
Зарплата           0.014910 -0.019093  1.000000    -0.030296   
Члены семьи       -0.008991 -0.006692 -0.030296     1.000000   
Страховые выплаты  0.010140  0.651030 -0.014963    -0.036290   

                   Страховые выплаты  
Пол                         0.010140  
Возраст                     0.651030  
Зарплата                   -0.014963  
Члены семьи                -0.036290  
Страховые выплаты           1.000000  


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответьте на вопрос и обоснуйте решение. 
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 a. Изменится. Приведите примеры матриц.
 b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** b. Не изменится. При умножении признаков на обратимую матрицу, параметры линейной регрессии также будут умножены на эту матрицу, что не изменит их отношение друг к другу. То есть, если в исходной задаче параметр a был в два раза больше параметра b, то после умножения признаков на матрицу это соотношение останется неизменным. Поэтому качество линейной регрессии не изменится.

**Обоснование:** 
Предположим, что мы умножаем матрицу признаков X на обратимую матрицу P, тогда

$X' = X * P$

вектор весов можно вычислить как:

$w' = ((X * P)^T * X * P)^{-1}* (X * P)^T * y$


раскроем скобки и воспользуемся свойствами транспонирования и свойствами обратной матрицы $(AB)^{-1} = B^{-1}A^{-1}$


$= P^{-1}((XP)^{T}X)^{-1} (XP)^Ty =$

$= P^{-1}((XP)^{T}X)^{-1} P^TX^Ty =$

$= P^{-1}(P^TX^TX)^{-1} P^TX^Ty =$

$= P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty$

умножение $(P^{T})^{-1} P^T = E$

$w' = P^{-1}(X^TX)^{-1} X^Ty$

$w' = P^{-1}w$

подставим значения в формулу для расчета предсказаний a':

$a'=X'w' = XPP^{-1}w = Xw = a$



## Алгоритм преобразования

**Алгоритм**

Алгоритм для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии с использованием обратимой матрицы Y:

1. Сгенерировать случайную матрицу Y размера (m, m), где m - количество признаков.

2. Проверить, что матрица Y обратима. Если не обратима, повторить шаг 1.

3. Умножить матрицу признаков X на матрицу Y

4. Найти коэффициенты регрессии для матрицы X' и предсказать значение целевой переменной для новых данных.

**Обоснование**

Умножение матрицы признаков на обратимую матрицу Y не изменяет качество модели регрессии. Это позволяет защитить информацию о признаках от злоумышленников, которые могут попытаться получить доступ к данным через коэффициенты регрессии или предсказанные значения целевой переменной. Обратимость матрицы Y необходима для того, чтобы можно было вернуться к исходным данным, если потребуется.

## Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

Исследуем качество модели без преобразования.

In [8]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [10]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_score = r2_score(target_test, np.round(model.predict(features_test),2))
print(R2_score)

0.4358187383756974


In [11]:
def transform_features(X):
    np.random.seed(12345)
    Y = np.random.rand(X.shape[1], X.shape[1])
    while np.linalg.det(Y) == 0:
        np.random.seed(12345)
        Y = np.random.rand(X.shape[1], X.shape[1])
    X_transformed = X.dot(Y)
    
    return X_transformed, Y

In [12]:
features, key = transform_features(features)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_score = r2_score(target_test, model.predict(features_test))
print(R2_score)

0.4352275712706788


R2 не изменился после преобразования.

Вывод:

Данные загружены и изучены, ошибок, дубликатов и пропусков нет.
Качество линейной регресии не изменилось при умножении признаков на обратимую матрицу.
Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним.

In [14]:
class Cipher:
    def __init__(self):
        self.key = None
        
    def transform_features(self, X):
        np.random.seed(12345)
        Y = np.random.rand(X.shape[1], X.shape[1])
        while np.linalg.det(Y) == 0:
            np.random.seed(12345)
            Y = np.random.rand(X.shape[1], X.shape[1])
        X_transformed = X.dot(Y)
        self.key = Y
        return X_transformed
    
    def decrypt(self, X):
        if self.key is None:
            raise ValueError("Key not found")
        Y_inv = np.linalg.inv(self.key)
        X_transformed = X.dot(Y_inv)
        return X_transformed

In [15]:
features.head()

,0,1,2,3
0,37169.983952,32441.905747,37126.682258,47708.455342
1,28484.575966,24863.156593,28457.829124,36560.358528
2,15743.503414,13742.238084,15729.981916,20206.383593
3,31241.345787,27266.582942,31200.559759,40101.520436
4,19563.289157,17075.165256,19542.546847,25108.599330


In [16]:
Cipher = Cipher()

In [17]:
new_f = Cipher.transform_features(features)
print(new_f.head())

              0             1             2             3
0  81176.508646  60423.457129  80137.811351  95800.305440
1  62214.092816  46309.769007  61418.780048  73422.188263
2  34387.011678  25596.474878  33947.376855  40581.644354
3  68225.069199  50782.794298  67352.480015  80517.040622
4  42726.508581  31803.459702  42179.585267  50422.926067


In [18]:
old_f = Cipher.decrypt(new_f)
print(old_f.head())

              0             1             2             3
0  37169.983952  32441.905747  37126.682258  47708.455342
1  28484.575966  24863.156593  28457.829124  36560.358528
2  15743.503414  13742.238084  15729.981916  20206.383593
3  31241.345787  27266.582942  31200.559759  40101.520436
4  19563.289157  17075.165256  19542.546847  25108.599330


Код работает. 